In [5]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [3]:
DATA_DIR="c:\\Users\\tihor\\Documents\\kaggle\\santander\\"
df_train=pd.read_csv(DATA_DIR+'train.csv')
df_test=pd.read_csv(DATA_DIR+'test.csv')

In [6]:
bayesian_tr_index,bayesian_val_index=list(StratifiedKFold(n_splits=2, shuffle=True, random_state=1).split(df_train, df_train.target.values))[0]

In [18]:
bayesian_tr_index.shape

(100000,)

In [19]:
bayesian_val_index.shape

(100000,)

In [31]:
import lightgbm as lgb
from sklearn import metrics
def LGB_bayesian(
    num_leaves,    #int
    min_data_in_leaf,    #int
    learning_rate,
    min_sum_hessian_in_leaf,    #int
    feature_fraction,
    lambda_l1,
    lambda_l2,
    min_gain_to_split,
    max_depth    #int
):
    num_leaves=int(num_leaves)
    min_data_in_leaf=int(min_data_in_leaf)
    max_depth=int(max_depth)
    param = {
        'num_leaves': num_leaves,
        'max_bin': 63,
        'min_data_in_leaf': min_data_in_leaf,
        'learning_rate': learning_rate,
        'min_sum_hessian_in_leaf': min_sum_hessian_in_leaf,
        'bagging_fraction': 1.0,
        'bagging_freq': 5,
        'feature_fraction': feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_gain_to_split': min_gain_to_split,
        'max_depth': max_depth,
        'save_binary': True, 
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,   

    }
    xg_train = lgb.Dataset(df_train.iloc[bayesian_tr_index][predictors].values,
                           label=df_train.iloc[bayesian_tr_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(df_train.iloc[bayesian_val_index][predictors].values,
                           label=df_train.iloc[bayesian_val_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )   

    num_round = 5000
    clf = lgb.train(param, xg_train, num_round, valid_sets = [xg_valid], verbose_eval=250, early_stopping_rounds = 50)
    predictions = clf.predict(df_train.iloc[bayesian_val_index][predictors].values, num_iteration=clf.best_iteration)
    score = metrics.roc_auc_score(df_train.iloc[bayesian_val_index][target].values, predictions)
    return score

In [32]:
# Bounded region of parameter space
bounds_LGB = {
    'num_leaves': (5, 20), 
    'min_data_in_leaf': (5, 20),  
    'learning_rate': (0.01, 0.3),
    'min_sum_hessian_in_leaf': (0.00001, 0.01),    
    'feature_fraction': (0.05, 0.5),
    'lambda_l1': (0, 5.0), 
    'lambda_l2': (0, 5.0), 
    'min_gain_to_split': (0, 1.0),
    'max_depth':(3,15),
}

In [33]:
from bayes_opt import BayesianOptimization
LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=13)

In [36]:
import warnings
init_points = 10
n_iter = 100
print('-' * 130)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

----------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_da... | min_ga... | min_su... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.881428
[500]	valid_0's auc: 0.88777
Early stopping, best iteration is:
[587]	valid_0's auc: 0.887978
|  11       |  0.888    |  0.1638   |  1.897    |  3.023    |  0.234    |  3.815    |  15.29    |  0.5483   |  0.001388 |  6.481    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.88189
Early stopping, best iteration is:
[303]	valid_0's auc: 0.882909
|  12       |  0.8829   |  0.1605   |  0.7589   |  4.63     |  0.2072   |  5.852    |  13.53    |  0.5566   |  0.000736 |  17

Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.883323
Early stopping, best iteration is:
[309]	valid_0's auc: 0.883655
|  27       |  0.8837   |  0.1691   |  0.1831   |  0.05318  |  0.1723   |  14.94    |  19.15    |  0.5242   |  0.009338 |  19.74    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.843961
[500]	valid_0's auc: 0.858545
[750]	valid_0's auc: 0.868061
[1000]	valid_0's auc: 0.875008
[1250]	valid_0's auc: 0.879987
[1500]	valid_0's auc: 0.883566
[1750]	valid_0's auc: 0.886166
[2000]	valid_0's auc: 0.888233
[2250]	valid_0's auc: 0.889852
[2500]	valid_0's auc: 0.890976
[2750]	valid_0's auc: 0.891922
[3000]	valid_0's auc: 0.892611
[3250]	valid_0's auc: 0.893052
[3500]	valid_0's auc: 0.893435
[3750]	valid_0's auc: 0.89375
[4000]	valid_0's auc: 0.893993
[4250]	valid_0's auc: 0.894176
Early stopping, best iteration is:
[4442]	valid_0's auc: 0.894279
|  28       |  0.8943   |  0.07121  |  4.623    |  0.6155 

[250]	valid_0's auc: 0.854518
[500]	valid_0's auc: 0.867195
[750]	valid_0's auc: 0.876007
[1000]	valid_0's auc: 0.881869
[1250]	valid_0's auc: 0.885873
[1500]	valid_0's auc: 0.888714
[1750]	valid_0's auc: 0.890568
[2000]	valid_0's auc: 0.891825
[2250]	valid_0's auc: 0.892596
[2500]	valid_0's auc: 0.893119
[2750]	valid_0's auc: 0.893592
[3000]	valid_0's auc: 0.893915
[3250]	valid_0's auc: 0.894072
Early stopping, best iteration is:
[3270]	valid_0's auc: 0.894102
|  40       |  0.8941   |  0.09048  |  4.481    |  4.585    |  0.02326  |  14.52    |  19.88    |  0.03173  |  0.003332 |  11.02    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.864553
[500]	valid_0's auc: 0.881965
[750]	valid_0's auc: 0.888542
[1000]	valid_0's auc: 0.891522
[1250]	valid_0's auc: 0.89263
[1500]	valid_0's auc: 0.892969
Early stopping, best iteration is:
[1659]	valid_0's auc: 0.893125
|  41       |  0.8931   |  0.1435   |  4.871    |  4.492    |  0.05195  |  14.96    |  5.3

Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.853724
[500]	valid_0's auc: 0.866001
[750]	valid_0's auc: 0.873336
[1000]	valid_0's auc: 0.878085
[1250]	valid_0's auc: 0.88171
[1500]	valid_0's auc: 0.88515
[1750]	valid_0's auc: 0.887438
[2000]	valid_0's auc: 0.8893
[2250]	valid_0's auc: 0.890706
[2500]	valid_0's auc: 0.89182
[2750]	valid_0's auc: 0.892608
[3000]	valid_0's auc: 0.893276
[3250]	valid_0's auc: 0.893667
[3500]	valid_0's auc: 0.893994
[3750]	valid_0's auc: 0.894238
[4000]	valid_0's auc: 0.894388
Early stopping, best iteration is:
[3975]	valid_0's auc: 0.894401
|  52       |  0.8944   |  0.05056  |  0.0431   |  1.728    |  0.02721  |  9.328    |  13.4     |  0.004708 |  0.004268 |  6.261    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.865995
[500]	valid_0's auc: 0.882002
[750]	valid_0's auc: 0.888004
[1000]	valid_0's auc: 0.890601
[1250]	valid_0's auc: 0.891639
Early stopping, best iteration is:
[

|  63       |  0.8941   |  0.07543  |  4.496    |  1.498    |  0.01683  |  7.529    |  12.57    |  0.00886  |  0.007142 |  8.633    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.826306
[500]	valid_0's auc: 0.855488
[750]	valid_0's auc: 0.869028
[1000]	valid_0's auc: 0.876768
[1250]	valid_0's auc: 0.881448
[1500]	valid_0's auc: 0.884664
[1750]	valid_0's auc: 0.887022
[2000]	valid_0's auc: 0.888748
[2250]	valid_0's auc: 0.89008
[2500]	valid_0's auc: 0.890896
[2750]	valid_0's auc: 0.891418
[3000]	valid_0's auc: 0.891692
Early stopping, best iteration is:
[3195]	valid_0's auc: 0.891832
|  64       |  0.8918   |  0.393    |  0.2925   |  4.806    |  0.01903  |  6.621    |  19.83    |  0.8681   |  0.00745  |  12.43    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.847013
[500]	valid_0's auc: 0.862037
[750]	valid_0's auc: 0.871443
[1000]	valid_0's auc: 0.877798
[1250]	valid_0's auc: 0.882343
[1500]	valid_0's auc: 

Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.825753
[500]	valid_0's auc: 0.840081
[750]	valid_0's auc: 0.849994
[1000]	valid_0's auc: 0.85838
[1250]	valid_0's auc: 0.86537
[1500]	valid_0's auc: 0.870409
[1750]	valid_0's auc: 0.874555
[2000]	valid_0's auc: 0.877739
[2250]	valid_0's auc: 0.880128
[2500]	valid_0's auc: 0.882248
[2750]	valid_0's auc: 0.883811
[3000]	valid_0's auc: 0.885234
[3250]	valid_0's auc: 0.886495
[3500]	valid_0's auc: 0.887571
[3750]	valid_0's auc: 0.888544
[4000]	valid_0's auc: 0.889259
[4250]	valid_0's auc: 0.889903
[4500]	valid_0's auc: 0.890508
[4750]	valid_0's auc: 0.891075
[5000]	valid_0's auc: 0.89147
Did not meet early stopping. Best iteration is:
[4998]	valid_0's auc: 0.891472
|  75       |  0.8915   |  0.07635  |  4.344    |  1.127    |  0.01627  |  3.102    |  19.22    |  0.08807  |  0.001908 |  19.78    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.867306
[500]	valid_0's auc

Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.873919
[500]	valid_0's auc: 0.885268
[750]	valid_0's auc: 0.889704
[1000]	valid_0's auc: 0.892071
[1250]	valid_0's auc: 0.893214
[1500]	valid_0's auc: 0.89353
Early stopping, best iteration is:
[1539]	valid_0's auc: 0.893591
|  86       |  0.8936   |  0.05543  |  4.688    |  0.1255   |  0.05415  |  14.85    |  13.96    |  0.05578  |  0.002553 |  11.92    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.871689
[500]	valid_0's auc: 0.883816
[750]	valid_0's auc: 0.889066
[1000]	valid_0's auc: 0.891398
[1250]	valid_0's auc: 0.89254
[1500]	valid_0's auc: 0.892847
Early stopping, best iteration is:
[1598]	valid_0's auc: 0.892951
|  87       |  0.893    |  0.08456  |  4.807    |  4.933    |  0.04601  |  14.87    |  19.87    |  0.4572   |  0.003995 |  15.98    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.843185
[500]	valid_0's auc:

Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.838783
[500]	valid_0's auc: 0.864019
[750]	valid_0's auc: 0.875384
[1000]	valid_0's auc: 0.881385
[1250]	valid_0's auc: 0.885115
[1500]	valid_0's auc: 0.887497
[1750]	valid_0's auc: 0.889235
[2000]	valid_0's auc: 0.890259
[2250]	valid_0's auc: 0.89104
[2500]	valid_0's auc: 0.891442
[2750]	valid_0's auc: 0.891839
Early stopping, best iteration is:
[2764]	valid_0's auc: 0.891858
|  97       |  0.8919   |  0.1686   |  0.06627  |  2.415    |  0.04176  |  3.062    |  5.436    |  0.2691   |  0.004902 |  14.69    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.841711
[500]	valid_0's auc: 0.866353
[750]	valid_0's auc: 0.877094
[1000]	valid_0's auc: 0.882953
[1250]	valid_0's auc: 0.886795
[1500]	valid_0's auc: 0.889297
[1750]	valid_0's auc: 0.890939
[2000]	valid_0's auc: 0.891915
[2250]	valid_0's auc: 0.892398
Early stopping, best iteration is:
[2436]	valid_0's auc: 0.8927

Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.876586
[500]	valid_0's auc: 0.888484
[750]	valid_0's auc: 0.891887
[1000]	valid_0's auc: 0.892464
Early stopping, best iteration is:
[971]	valid_0's auc: 0.892547
|  108      |  0.8925   |  0.07761  |  0.5533   |  4.945    |  0.1083   |  14.92    |  7.254    |  0.9657   |  0.008522 |  7.81     |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.838913
[500]	valid_0's auc: 0.85534
[750]	valid_0's auc: 0.865078
[1000]	valid_0's auc: 0.872287
[1250]	valid_0's auc: 0.877607
[1500]	valid_0's auc: 0.881183
[1750]	valid_0's auc: 0.884018
[2000]	valid_0's auc: 0.886126
[2250]	valid_0's auc: 0.887735
[2500]	valid_0's auc: 0.889156
[2750]	valid_0's auc: 0.890106
[3000]	valid_0's auc: 0.890834
[3250]	valid_0's auc: 0.89142
[3500]	valid_0's auc: 0.891986
[3750]	valid_0's auc: 0.892437
[4000]	valid_0's auc: 0.892833
[4250]	valid_0's auc: 0.893181
[4500]	valid_0's auc: 0.893442
[47

[1500]	valid_0's auc: 0.891414
[1750]	valid_0's auc: 0.892555
[2000]	valid_0's auc: 0.89328
[2250]	valid_0's auc: 0.893775
[2500]	valid_0's auc: 0.893957
Early stopping, best iteration is:
[2672]	valid_0's auc: 0.8941
|  118      |  0.8941   |  0.06083  |  0.02689  |  2.318    |  0.03382  |  14.95    |  7.408    |  0.9      |  0.007813 |  9.257    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.86614
[500]	valid_0's auc: 0.8769
[750]	valid_0's auc: 0.882509
[1000]	valid_0's auc: 0.886836
[1250]	valid_0's auc: 0.889404
[1500]	valid_0's auc: 0.891314
[1750]	valid_0's auc: 0.892585
[2000]	valid_0's auc: 0.893372
[2250]	valid_0's auc: 0.893819
Early stopping, best iteration is:
[2250]	valid_0's auc: 0.893819
|  119      |  0.8938   |  0.08983  |  3.492    |  1.817    |  0.02153  |  14.75    |  5.108    |  0.3047   |  0.000364 |  19.87    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.853819
[500]	valid_0's auc: 

In [38]:
print(LGB_BO.max['target'])
LGB_BO.max['params']

0.8947079500329543


{'feature_fraction': 0.05417588475081639,
 'lambda_l1': 4.960965942297994,
 'lambda_l2': 1.3793048534309067,
 'learning_rate': 0.027674233178344776,
 'max_depth': 14.878429109614494,
 'min_data_in_leaf': 9.248155828857863,
 'min_gain_to_split': 0.04245137968559498,
 'min_sum_hessian_in_leaf': 0.0041135185671155455,
 'num_leaves': 11.648350230450859}

In [1]:
#just probe for specific params
LGB_BO.probe(
    params={'feature_fraction': 0.1403, 
            'lambda_l1': 4.218, 
            'lambda_l2': 1.734, 
            'learning_rate': 0.07, 
            'max_depth': 14, 
            'min_data_in_leaf': 17, 
            'min_gain_to_split': 0.1501, 
            'min_sum_hessian_in_leaf': 0.000446, 
            'num_leaves': 6},
    lazy=True, # 
)
LGB_BO.maximize(init_points=0, n_iter=0) # remember no init_points or n_iter
print(LGB_BO.max['target'])
LGB_BO.max['params']